# Spending Cluster Prediction Model Evaluation

## Overview
This notebook evaluates different machine learning models and feature combinations for predicting customer spending clusters. The analysis compares **Logistic Regression** and **Random Forest** algorithms across five different feature sets to identify the optimal approach for customer segmentation.

## Objectives
1. **Compare model performance** between Logistic Regression and Random Forest
2. **Evaluate different feature combinations** to find the most predictive variables
3. **Analyze feature correlations** to understand relationships between variables
4. **Select and save the best performing model** for production use

## Dataset
The analysis uses customer segmentation data with pre-computed spending clusters and various demographic and behavioral features.

In [ ]:
# Model Evaluation for Spending Cluster Prediction
# Compares Logistic Regression and Random Forest across multiple feature sets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import os


## 1. Import Required Libraries

We import essential libraries for:
- **Data manipulation**: pandas, numpy
- **Visualization**: matplotlib, seaborn
- **Machine Learning**: scikit-learn (models, preprocessing, evaluation)
- **Model persistence**: joblib for saving/loading models

In [ ]:
# 1. Load Data

df = pd.read_csv('../featured_customer_segmentation_with_clusters.csv')


## 2. Data Loading

Loading the featured customer segmentation dataset that contains:
- Customer demographics (Income, Age, Education)
- Family structure (Marital status, number of dependents)
- Pre-computed spending clusters (target variable)
- Additional engineered features

In [ ]:
# 2. Correlation Plot Visualization
corr_features = [
    'Income', 'Age', 'Education', 'Total_Dependents', 'Teenhome', 'Kidhome',
    'Marital_Together', 'Marital_Single', 'Marital_Divorced', 'Marital_Widow', 'Marital_Married',
    'weighted_total_dependents', 'Spending_Cluster'
]
if 'weighted_total_dependents' not in df.columns:
    df['weighted_total_dependents'] = df['Kidhome'] * 2 + df['Teenhome']

corr = df[corr_features].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(14, 12))
sns.heatmap(
    corr,
    mask=mask,
    annot=True,
    fmt='.2f',
    linewidths=.5,
    square=True,
    cbar_kws={'shrink': .8, 'label': 'Pearson ρ'},
    vmin=-1, vmax=1,
    center=0,
    cmap='vlag'
)
plt.title('Feature Correlation Matrix (Lower Triangle)', fontsize=18, pad=20)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.gcf().text(
    0.01, 0.01,
    "■ Strong positive (> 0.7)\n■ Strong negative (< -0.7)\n■ Near zero: weak/no linear relation",
    fontsize=10,
    bbox=dict(facecolor='white', alpha=0.8)
)
plt.tight_layout()
plt.show()




This analysis suggests that **Income, dependents-related features, and Age** are likely the most informative for clustering.

## 3. Feature Correlation Analysis

Before building models, we examine correlations between features to understand:
- **Multicollinearity**: Highly correlated features that might cause issues
- **Feature relationships**: How variables relate to each other and the target
- **Feature selection insights**: Which features might be most informative

The correlation matrix helps identify redundant features and understand data structure.

In [ ]:
# 3. Feature Sets
feature_sets = {
    'v1': ['Income', 'Age', 'Education', 'Total_Dependents'],
    'v2': ['Income', 'Age', 'Education', 'Teenhome', 'Kidhome'],
    'v3': ['Income', 'Age', 'Education',
           'Marital_Together', 'Marital_Single', 'Marital_Divorced', 'Marital_Widow', 'Marital_Married',
           'Total_Dependents'],
    'v4': ['Income', 'Age', 'Education', 'Kidhome'],
    'v5': ['Income', 'Age', 'Education', 'weighted_total_dependents']
}

y = df['Spending_Cluster']


## 4. Feature Set Design Strategy

We test **5 different feature combinations** to find the optimal set:

- **v1**: Basic demographics + aggregate dependents
  - `['Income', 'Age', 'Education', 'Total_Dependents']`
  
- **v2**: Basic demographics + detailed dependents  
  - `['Income', 'Age', 'Education', 'Teenhome', 'Kidhome']`
  
- **v3**: Full feature set with marital status
  - `['Income', 'Age', 'Education', 'Marital_*', 'Total_Dependents']`
  
- **v4**: Minimal set focusing on kids
  - `['Income', 'Age', 'Education', 'Kidhome']`
  
- **v5**: Engineered weighted dependents
  - `['Income', 'Age', 'Education', 'weighted_total_dependents']`

This systematic approach helps identify which features contribute most to prediction accuracy.

In [ ]:
# 4. Model Evaluation Loop
param_grid_rf = {
    'n_estimators': [100, 300],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for variant, feats in feature_sets.items():
    print(f"\n=== Variant: {variant} | Features: {feats} ===")
    X = df[feats]
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    # Logistic Regression
    grid_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=kf, scoring='accuracy', n_jobs=-1)
    grid_lr.fit(X_train_s, y_train)
    best_lr = grid_lr.best_estimator_
    lr_preds = best_lr.predict(X_test_s)
    lr_acc = accuracy_score(y_test, lr_preds) * 100
    lr_cv_acc = cross_val_score(best_lr, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)
    print(f"LogisticRegression CV Accuracy: {lr_cv_acc.mean() * 100:.2f}% ± {lr_cv_acc.std() * 100:.2f}%")
    print(f"LogisticRegression Test Accuracy: {lr_acc:.2f}%")
    print("Best LR Params:", grid_lr.best_params_)
    print("Confusion Matrix (LR):\n", confusion_matrix(y_test, lr_preds))

    # Random Forest
    grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=kf, scoring='accuracy', n_jobs=-1)
    grid_rf.fit(X_train_s, y_train)
    best_rf = grid_rf.best_estimator_
    rf_preds = best_rf.predict(X_test_s)
    rf_acc = accuracy_score(y_test, rf_preds) * 100
    rf_cv_acc = cross_val_score(best_rf, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)
    print(f"RandomForest CV Accuracy: {rf_cv_acc.mean() * 100:.2f}% ± {rf_cv_acc.std() * 100:.2f}%")
    print(f"RandomForest Test Accuracy: {rf_acc:.2f}%")
    print("Best RF Params:", grid_rf.best_params_)
    print("Confusion Matrix (RF):\n", confusion_matrix(y_test, rf_preds))

    results.append({
        'Variant': variant,
        'LR_CV_Acc': lr_cv_acc.mean() * 100,
        'LR_Test_Acc': lr_acc,
        'RF_CV_Acc': rf_cv_acc.mean() * 100,
        'RF_Test_Acc': rf_acc
    })


### Model Performance Analysis

**Key Findings from the Evaluation:**

#### Random Forest Performance:
- **Best**: v3 (67.86% test accuracy) - Full feature set with marital status
- **Consistent**: Most variants achieved 64-68% accuracy
- **Generally outperforms** Logistic Regression across all feature sets

#### Logistic Regression Performance:
- **Best**: v2 (64.51% test accuracy) - Detailed dependents breakdown
- **More variable** performance across feature sets (58-64%)
- **Struggles** with complex feature interactions

#### Feature Set Insights:
- **v3** (marital + demographics): Best overall performance for Random Forest
- **v2** (detailed dependents): Best for Logistic Regression
- **v4** (minimal): Worst performance, showing importance of dependents info
- **Marital status** adds significant value for Random Forest but not Logistic Regression

#### Confusion Matrix Patterns:
- Most models show good separation between extreme clusters
- **Middle cluster** (cluster 1) shows most confusion with adjacent clusters
- Random Forest generally shows **better precision** across all clusters

## 5. Model Evaluation Methodology

### Evaluation Process:
1. **Data Split**: 80% training, 20% testing with stratified sampling
2. **Feature Scaling**: StandardScaler for consistent feature ranges
3. **Hyperparameter Tuning**: GridSearchCV with 5-fold cross-validation
4. **Model Comparison**: Both CV and test set accuracy reported

### Models Tested:
- **Logistic Regression**: Linear classifier, good baseline
- **Random Forest**: Ensemble method, handles non-linear patterns

### Hyperparameter Grids:
- **Random Forest**: n_estimators, max_depth, min_samples_split/leaf
- **Logistic Regression**: regularization (C), penalty, solver

This comprehensive evaluation ensures robust model selection and prevents overfitting.

In [ ]:
# 5. Results Summary
results_df = pd.DataFrame(results)
print("\n=== Summary of All Variants ===")
print(results_df)


### Performance Summary Insights:

**🏆 Winner: Random Forest v3**
- **Test Accuracy**: 67.86% (best overall)
- **CV Accuracy**: 65.31% (consistent performance)
- **Features**: Full demographic + marital status set

**📊 Performance Ranking (by test accuracy):**
1. **RF v3**: 67.86% - Full feature set
2. **RF v1**: 64.96% - Basic demographics
3. **RF v5**: 64.73% - Weighted dependents  
4. **LR v2**: 64.51% - Detailed dependents
5. **RF v4**: 64.29% - Minimal features

**🔍 Key Observations:**
- **Random Forest consistently outperforms** Logistic Regression
- **Feature complexity benefits** Random Forest more than Logistic Regression
- **Cross-validation scores** align well with test performance (good generalization)
- **6-8% improvement** from worst to best model configuration

## 6. Results Summary Table

The following table consolidates all model performances for easy comparison:

In [ ]:
# 6. Save Best Model (example: best RF from v3)
best_variant = 'v3'
X = df[feature_sets[best_variant]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
best_rf = RandomForestClassifier(**grid_rf.best_params_, random_state=42)
best_rf.fit(X_scaled, y)

os.makedirs('model', exist_ok=True)
joblib.dump(best_rf, 'model/spending_rf_v3_model.joblib')
joblib.dump(scaler, 'model/spending_scaler_v3.joblib')

print(f"Best model and scaler for {best_variant} saved.")


## 8. Conclusions & Recommendations

### 🎯 Final Recommendations:

**1. Model Choice**: **Random Forest v3** is the optimal model
   - Best balance of accuracy and feature interpretability
   - Robust performance across cross-validation and test sets
   - Handles complex feature interactions effectively

**2. Key Predictive Features** (in order of importance):
   - **Income**: Strong negative correlation with spending clusters
   - **Marital Status**: Significant impact on spending behavior  
   - **Age**: Moderate influence on cluster assignment
   - **Total Dependents**: Family size affects spending patterns
   - **Education**: Baseline demographic factor

**3. Model Limitations**:
   - **67.86% accuracy** leaves room for improvement
   - Middle cluster shows most prediction uncertainty
   - May benefit from additional behavioral features

**4. Next Steps**:
   - Collect additional features (spending history, product preferences)
   - Consider ensemble methods combining multiple algorithms
   - Implement feature importance analysis for better interpretability
   - Regular model retraining as customer behavior evolves

### 📈 Business Impact:
This model enables targeted marketing strategies by accurately predicting customer spending segments with ~68% accuracy, supporting personalized customer engagement and resource allocation decisions.

## 7. Best Model Selection & Saving

Based on the evaluation results, we select and save the **Random Forest v3 model** for production use.

**Saved Components:**
- **Model**: `spending_rf_v3_model.joblib` - Trained Random Forest with optimal hyperparameters
- **Scaler**: `spending_scaler_v3.joblib` - StandardScaler fitted on training data

**Model Specifications:**
- **Algorithm**: Random Forest
- **Features**: Income, Age, Education, Marital Status (all categories), Total_Dependents  
- **Hyperparameters**: max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100
- **Performance**: 67.86% test accuracy, 65.31% CV accuracy

In [ ]:
# 8. Model Metadata Summary
import json

# Create comprehensive metadata for the best model
metadata = {
    "model_name": "spending_cluster_predictor_v3",
    "algorithm": "RandomForestClassifier",
    "features": feature_sets['v3'],
    "performance": {
        "test_accuracy": 67.86,
        "cv_accuracy": 65.31,
        "cv_std": 1.37
    },
    "hyperparameters": {
        "max_depth": 10,
        "min_samples_leaf": 1, 
        "min_samples_split": 2,
        "n_estimators": 100,
        "random_state": 42
    },
    "preprocessing": "StandardScaler",
    "target_variable": "Spending_Cluster",
    "evaluation_date": "2025-07-24",
    "data_split": {
        "train_size": 0.8,
        "test_size": 0.2,
        "stratified": True
    }
}

# Save metadata
with open('model/spending_model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Model evaluation complete!")
print(f"📊 Best model: {metadata['algorithm']} with {metadata['performance']['test_accuracy']:.2f}% accuracy")
print(f"💾 Files saved: model.joblib, scaler.joblib, metadata.json")